# Extra

This notebook contains extra contents for managing the MFG ML demo solution.

## Before you begin

### Set your project ID

If you don't know your project ID, you may be able to get your project ID using gcloud.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]

print("Project ID:", PROJECT_ID)

## Create and Configure Service Account for Scheduled Notebook Execution

When [scheduling a managed notebook execution](https://cloud.google.com/vertex-ai/docs/workbench/managed/schedule-managed-notebooks-run-quickstart), you can select a service account to execute the managed notebook as. The service account must have the following role assignments:

1. `roles/notebooks.runner`
1. `roles/storage.objectAdmin`

In this case, since our notebook require access to BigQuery and Vertex AI, you will have to grant the service account these roles as well:

1. `roles/bigquery.dataEditor`
1. `roles/bigquery.jobUser`
1. `roles/bigquery.readSessionUser`
1. `roles/aiplatform.user`

### Prerequisites

In order to run cells in this section, your user account or service account must have the following roles: 

1. `roles/iam.serviceAccountCreator`

In [ ]:
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

print("SERVICE_ACCOUNT:", SERVICE_ACCOUNT)

In [ ]:
SA_NOTEBOOK = "vertex-notebook-runner"

In [ ]:
! gcloud iam service-accounts create $SA_NOTEBOOK \
--description="Vertex AI managed notebook runner service account for scheduled execution" \
--display-name="Vertex AI Notebook Runner Service Account"

In [ ]:
! gcloud projects add-iam-policy-binding {PROJECT_ID} \
--member="serviceAccount:{SA_NOTEBOOK}@{PROJECT_ID}.iam.gserviceaccount.com" \
--role='roles/notebooks.runner' > /dev/null

! gcloud projects add-iam-policy-binding {PROJECT_ID} \
--member="serviceAccount:{SA_NOTEBOOK}@{PROJECT_ID}.iam.gserviceaccount.com" \
--role='roles/storage.objectAdmin' > /dev/null

! gcloud projects add-iam-policy-binding {PROJECT_ID} \
--member="serviceAccount:{SA_NOTEBOOK}@{PROJECT_ID}.iam.gserviceaccount.com" \
--role='roles/bigquery.dataEditor' > /dev/null

! gcloud projects add-iam-policy-binding {PROJECT_ID} \
--member="serviceAccount:{SA_NOTEBOOK}@{PROJECT_ID}.iam.gserviceaccount.com" \
--role='roles/bigquery.jobUser' > /dev/null

! gcloud projects add-iam-policy-binding {PROJECT_ID} \
--member="serviceAccount:{SA_NOTEBOOK}@{PROJECT_ID}.iam.gserviceaccount.com" \
--role='roles/bigquery.readSessionUser' > /dev/null

! gcloud projects add-iam-policy-binding {PROJECT_ID} \
--member="serviceAccount:{SA_NOTEBOOK}@{PROJECT_ID}.iam.gserviceaccount.com" \
--role='roles/aiplatform.user' > /dev/null

## [Schedule a managed notebook run](https://cloud.google.com/vertex-ai/docs/workbench/managed/schedule-managed-notebooks-run-quickstart#schedule_an_execution)

**Limitation**: Currently, you can only schedule a managed notebook run through the JupyterLab or GCP console interface. This may change in the future.

## Clean Up

Programmatically delete BigQuery views used for batch predictions.

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=PROJECT_ID)

In [ ]:
query = f"""
SELECT
    table_name
FROM 
    `{PROJECT_ID}.ml.INFORMATION_SCHEMA.VIEWS`
WHERE STARTS_WITH(table_name, 'prediction_features');
"""

results = bq_client.query(query).to_dataframe()

results

In [ ]:
tables = results.to_dict()["table_name"]

for table in tables.values():
    query = f"""DROP VIEW ml.{table};"""
    results = bq_client.query(query)